# **Event Camera Simulator**

In [1]:
import cv2
import numpy as np
import simulator_utils as camera

In [11]:
from skimage import data
import matplotlib.pyplot as plt

image1 = data.astronaut()
image2 = np.clip(image1 + np.random.normal(3, 10, image1.shape), 0, 255).astype(np.uint8)
image3 = np.clip(image2 + np.random.normal(3, 10, image2.shape), 0, 255).astype(np.uint8)
image4 = np.clip(image3 - np.random.normal(3, 10, image3.shape), 0, 255).astype(np.uint8)
image5 = np.clip(image4 - np.random.normal(3, 10, image4.shape), 0, 255).astype(np.uint8)
images = np.array([image1, image2, image3, image4, image5])


def create_moving_square_video(direction, video_path, duration=3, fps=30, square_size=200):
    # Video parameters
    width, height = 640, 480
    total_frames = duration * fps
    # Create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_path, fourcc, fps, (width, height))
    frames = []
    # Create frames with a moving square
    for frame_idx in range(total_frames):
        # Create a black background
        frame = np.zeros((height, width, 3), dtype=np.uint8)
        # Calculate the position of the square
        if direction == 0:
            x_pos = int((frame_idx / total_frames) * (width - square_size))
            y_pos = (height - square_size) // 2
        elif direction == 1:
            x_pos = (width - square_size) - int((frame_idx / total_frames) * (width - square_size))
            y_pos = (height - square_size) // 2
        elif direction == 2:
            y_pos = int((frame_idx / total_frames) * (height - square_size))
            x_pos = (width - square_size) // 2
        else:
            y_pos = (height - square_size) - int((frame_idx / total_frames) * (height - square_size))
            x_pos = (width - square_size) // 2
        # Draw the square
        frame[y_pos:y_pos + square_size, x_pos:x_pos + square_size] = (255, 255, 255)
        # Write the frame to the video
        frames.append(frame)
        out.write(frame)
    # Release the VideoWriter object
    out.release()
    return frames

# Path to save the video
video_path = 'moving_square.mp4'
# Create the video
frames = create_moving_square_video(3, video_path)
print(f"Video saved at {video_path}")

Video saved at moving_square.mp4


#### **RGB Frames to Event Frames**
This methos converts an RGB video frame from a conventional framing camera to event frames. 
- Convert RGB frame to a grayscale frame.
- Converts grayscale frame to linear log frame.
- Applies a low-pass filter based on intensity of frame to reproduce the band-pass filter nature of individual pixel circuit.
- The low-pass frame is compared with the previous memorised frame to compute an event frame.

In [12]:
video_fps = 30
sampling_period = 1/video_fps
cutoff_freq = 300
pos_thresh, neg_thresh = 0.6, 0.6
pos_frames, neg_frames, evts_video = camera.event_video(frames, sampling_period, cutoff_freq, pos_thresh, neg_thresh)

Hello!
Min difference any pixel: 0.0
Max difference any pixel: 0.0
Max events any pixel: 0
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min difference any pixel: -5.541263580322266
Max difference any pixel: 5.541263580322266
Max events any pixel: 9
Min d

In [13]:
def save_video_from_tensor(tensor, output_path, fps=30):
    num_frames, height, width = tensor.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height), False)
    for i in range(num_frames):
        frame = tensor[i]
        out.write(frame)
    out.release()

output_path = 'moving_square_tensor.mp4'
save_video_from_tensor(evts_video, output_path)
print(f"Video saved at {output_path}")

Video saved at moving_square_tensor.mp4


#### **Event Frames to Event List**
Converts an RGB frame into an events list, which is $N\times4$ matrtix. The columns of the matrix are:
- Column 1: Time of event.
- Column 2: $x$ address of triggered pixel in frame.
- Column 3: $y$ address of triggered pixel in frame.
- Column 4: Polarity of triggered pixel in frame.

In [4]:
event_list = camera.event_list(pos_frames, neg_frames, sampling_period)

#### **Event List to Event Frames**
Converts an events list back to event frames by producing a histogram and normalising it.

In [5]:
frames = camera.events_list_frames(event_list, 512, 512, sampling_period)